In [ ]:
import torch
from torchtext import data
from torchtext import datasets
import random
import numpy as np

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

torch.cuda.is_available()

True

In [ ]:
!pip install transformers -qq
!pip install sentencepiece -qq
!pip install numpy requests nlpaug

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 59.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 106.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 86.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 5.4 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##Load Dataset

In [ ]:
import random
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw

from nlpaug.util import Action

In [ ]:
SEED = 1234

In [ ]:
alpha_common_error = 0.10
alpha_common_error_char = 0.05
aug1_OCR = nac.OcrAug(aug_word_p=alpha_common_error)
aug2_Rins = nac.RandomCharAug(action="insert", aug_word_p=alpha_common_error, aug_char_min=1, aug_char_max=1, aug_char_p=alpha_common_error_char)
aug3_Rsub = nac.RandomCharAug(action="substitute", aug_word_p=alpha_common_error, aug_char_min=1, aug_char_max=1, aug_char_p=alpha_common_error_char)
aug4_Rswa = nac.RandomCharAug(action="swap", aug_word_p=alpha_common_error,aug_char_min=1, aug_char_max=1, aug_char_p=alpha_common_error_char) #
aug5_Rdel = nac.RandomCharAug(action="delete", aug_word_p=alpha_common_error, aug_char_min=1, aug_char_max=1, aug_char_p=alpha_common_error_char)
aug6_Kb = nac.KeyboardAug(aug_word_p=alpha_common_error)
aug7_Split = naw.SplitAug(aug_p=alpha_common_error)

In [ ]:
random.sample(range(0, 7), 2 - 1)

[3]

In [ ]:
def text2augment(text, m):
    output = [text, ]

    temp = random.sample(range(0, 7), m - 1)

    if 0 in temp:
        output.append(*aug1_OCR.augment(text))
    if 1 in temp:
        output.append(*aug2_Rins.augment(text))
    if 2 in temp:
        output.append(*aug3_Rsub.augment(text))
    if 3 in temp:
        output.append(*aug4_Rswa.augment(text))
    if 4 in temp:
        output.append(*aug5_Rdel.augment(text))
    if 5 in temp:
        output.append(*aug6_Kb.augment(text))
    if 6 in temp:
        output.append(*aug7_Split.augment(text))

    return output

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
def to_df(x, y):
    d = {"text": x, "label": y}
    return pd.DataFrame(d)

def split_3(df, test_size=0.2, valid_size=0.2):
    _df = df.copy().sample(frac=1).reset_index()
    _df = _df[["text", "label"]]

    x = _df["text"].copy()
    y = _df["label"].copy()
    #split train-test
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_size, stratify=y,random_state=SEED)
    # split train-valid
    x, y = x_train, y_train
    x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size=valid_size, stratify=y,random_state=SEED)
    return to_df(x_train, y_train), to_df(x_valid, y_valid), to_df(x_test, y_test)

In [ ]:
from itertools import chain

In [ ]:
def aug_replicate(y_labels):
    #return list(chain(* [[y]*(2 if y == 0 else 4) for y in y_labels] ))
    return list(chain(* [[y]*(1 if y == 0 else 3) for y in y_labels] ))
    #return list(chain(* [[y]*(1 if y == 0 else 2) for y in y_labels] ))
    #return list(chain(* [[y]*(2 if y == 0 else 3) for y in y_labels] ))


def aug_text(x_text, y_labels):
    #x_text = [ text2augment(x, 2 if y == 0 else 4) for x, y in zip(x_text, y_labels)]
    x_text = [ text2augment(x, 1 if y == 0 else 3) for x, y in zip(x_text, y_labels)]
    #x_text = [ text2augment(x, 1 if y == 0 else 2) for x, y in zip(x_text, y_labels)]
    #x_text = [ text2augment(x, 2 if y == 0 else 3) for x, y in zip(x_text, y_labels)]
    return pd.Series(list(chain(*x_text)), index=None)

def split_3_aug(df, test_size=0.2, valid_size=0.2):
    _df = df.copy().sample(frac=1).reset_index()
    _df = _df[["text", "label"]]

    x = _df["text"].copy()
    y = _df["label"].copy()
    #split train-test
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_size, stratify=y,random_state=SEED)
    # augment
    # x_test = aug_text(x_test, y_test)
    # y_test = aug_replicate(y_test)
    # split train-valid
    x, y = x_train, y_train
    x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size=valid_size, stratify=y,random_state=SEED)
    # augment
    x_train = aug_text(x_train, y_train)
    y_train = aug_replicate(y_train)
    x_valid = aug_text(x_valid, y_valid)
    y_valid = aug_replicate(y_valid)

    print(x_valid.shape)
    print("DONE")
    print(len(y_valid))

    print(x_train.shape)
    print("DONE")
    print(len(y_train))

    return to_df(x_train, y_train), to_df(x_valid, y_valid), to_df(x_test, y_test)

In [ ]:
import numpy as np
import pandas as pd

tname_data = "/content/drive/MyDrive/TP1-AntiDiscriminator/datasets/final/hsd_merge_cleaned_lowered"
data = pd.read_csv(f"{tname_data}.csv")

train, valid, test = split_3(data)

X_train = train['text']
y_train = train['label']

X_valid = valid['text']
y_valid = valid['label']

X_test = test['text']
y_test = test['label']

In [ ]:
# temp = "/content/drive/MyDrive/TP1-AntiDiscriminator/datasets/final/augmented/hsd_merge_aug_train.csv"
# train.to_csv(temp,  sep=",", index=False)

# temp = "/content/drive/MyDrive/TP1-AntiDiscriminator/datasets/final/augmented/hsd_merge_aug_valid.csv"
# valid.to_csv(temp,  sep=",", index=False)

# temp = "/content/drive/MyDrive/TP1-AntiDiscriminator/datasets/final/augmented/hsd_merge_aug_test.csv"
# test.to_csv(temp,  sep=",", index=False)

In [ ]:
data[data["label"] == 1]

,text,label
0,pillaron al presunto es de nacionalidad colomb...,1
1,cuando se den cuenta que eran homicidas o narc...,1
2,jajaajja ahí estuviste bien chouza se nota tu ...,1
3,acá podemos ver a un alemán de cabello rubio o...,1
4,chile murieron 8 compatriotas por culpa de un ...,1
...,...,...
17778,marinovic hay alguna mujer del movimiento femi...,1
17782,leí tu descripción ahora entiendo todo el daño...,1
17816,🤮🤮🤮🤮una 💩 de debate solo orrego se presta para...,1
17822,los delincuentes de la primers linea queman ig...,1


In [ ]:
train[train["label"] == 1]

,text,label
7757,que puta pereza me da puigdemont chico cómprat...,1
9942,me gustaría saber cuantos extranjeros ilegales...,1
4282,te digo yo donde te puedes sentar furcia,1
5002,yisus pucha se que te hubiese encantado ser es...,1
580,ya es dia 9 mira eres imbécil un imbécil muy l...,1
...,...,...
17710,inmigrante fue detenido por ser asesino de 8 p...,1
12549,lrt diego brando es gay me cago en la puta,1
11913,se cumple el sueño de que todos te metemos la ...,1
7792,hasta el cuñado inés arimadas la llama facha y...,1


In [ ]:
valid[valid["label"] == 1]

,text,label
14327,son unos sinvergüenzas no más subvenciones a b...,1
8553,pasó a principios de mes su delito llamarse al...,1
6019,este negro de mierda mirándole las cosas a mi ...,1
17427,los zurdos son una lagra qué no debe existir t...,1
11400,sea cómo sea los comunistas son pedófilos y la...,1
...,...,...
17203,muy ateos que sois pero bien que os gustaría s...,1
5294,ghostt subnormal se nota a la legua que eres t...,1
8216,menuda gilipollez de comentario estás un poqui...,1
8755,lucha de consejos cde busca frenar orden del c...,1


In [ ]:
test[test["label"] == 1]

,text,label
11309,el turismo de patera no parece que deje muchos...,1
5424,dale la vuelta a la camiseta y tienes el 6 del...,1
15574,a las caretas beatas solo merecen insultos y c...,1
8279,mi profe tiene como una voz colombiana venezol...,1
16352,pero vamos a ver hijos de la grandísima puta ¿...,1
...,...,...
12853,si llevas a tu hijo menor en el coche sin cint...,1
12337,te voy a reventar las putas piernas con el par...,1
5763,qué tiene que hacer un inmigrante en nuestra c...,1
1571,landauri negro de mierda fracasado,1


In [ ]:

# import numpy as np
# import pandas as pd

# tname_data = "/content/drive/MyDrive/TP1-AntiDiscriminator/datasets/original/dataset_9834_cleaned_lowered"
# data = pd.read_csv(f"{tname_data}.csv")
# # test = pd.read_csv(f"{tname_data}_test.csv")

# data = data.sample(frac=1).reset_index()
# data = data[["text", "label"]]

# train = data[: int(len(data) * 0.7)]
# valid = data[int(len(data) * 0.7) : int(len(data) * 0.9)]
# test = data[int(len(data) * 0.9) :]

# X_train = train['text']
# y_train = train['label']

# X_valid = valid['text']
# y_valid = valid['label']

# X_test = test['text']
# y_test = test['label']

In [ ]:

# import numpy as np
# import pandas as pd

# tname_data = "hsd_merged_AUG"

# data = pd.read_csv(f"{tname_data}_no_test.csv")
# test = pd.read_csv(f"{tname_data}_test.csv")


# train = data[: int(len(data) * 0.9)]
# valid = data[int(len(data) * 0.9) :]

# X_train = train['text']
# y_train = train['label']

# X_valid = valid['text']
# y_valid = valid['label']

# X_test = test['text']
# y_test = test['label']


## Set Cuda

In [ ]:
# import torch
# from torchtext import data
# from torchtext import datasets
# import random
# import numpy as np

# SEED = 1234

# random.seed(SEED)
# np.random.seed(SEED)
# torch.manual_seed(SEED)
# torch.backends.cudnn.deterministic = True

# torch.cuda.is_available()

# Extract feature by using BETO

In [ ]:
#train_sentences

In [ ]:
import pandas as pd
from glob import glob

train_sentences = list(train['text'].values)
train_labels = list(train['label'].values)

valid_sentences = list(valid['text'].values)
valid_labels = list(valid['label'].values)

test_sentences = list(test['text'].values)
test_labels = list(test['label'].values)

In [ ]:
test_labelsdf = pd.DataFrame(test_labels)
test_labelsdf.to_csv('test_labels.csv',index=False)

Load tokenizer of BETO

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('dccuchile/bert-base-spanish-wwm-cased')

In [ ]:
tokenizer.vocab_size

31002

In [ ]:
#choose max_length for beto model based on the input length

max_length = 0
list_len=[]
for sentence in train_sentences:
    length = len(tokenizer.tokenize(sentence))
    list_len.append(length)

from collections import Counter
Counter(list_len).most_common(100)

[(17, 374),
 (21, 371),
 (15, 363),
 (18, 356),
 (22, 342),
 (12, 342),
 (14, 340),
 (24, 336),
 (23, 331),
 (13, 329),
 (19, 327),
 (16, 323),
 (25, 317),
 (26, 315),
 (20, 306),
 (11, 300),
 (10, 299),
 (27, 282),
 (9, 274),
 (29, 253),
 (28, 251),
 (30, 215),
 (31, 207),
 (8, 206),
 (7, 205),
 (32, 198),
 (33, 191),
 (6, 147),
 (34, 142),
 (36, 132),
 (35, 125),
 (43, 119),
 (49, 117),
 (38, 116),
 (53, 116),
 (40, 115),
 (37, 114),
 (41, 113),
 (5, 112),
 (50, 111),
 (39, 108),
 (44, 108),
 (54, 107),
 (46, 107),
 (48, 103),
 (45, 101),
 (47, 100),
 (42, 98),
 (51, 97),
 (52, 97),
 (56, 88),
 (57, 85),
 (55, 79),
 (58, 71),
 (4, 70),
 (59, 65),
 (60, 58),
 (61, 55),
 (62, 43),
 (64, 37),
 (63, 32),
 (3, 32),
 (65, 29),
 (66, 26),
 (67, 20),
 (68, 17),
 (69, 14),
 (2, 12),
 (72, 6),
 (71, 3),
 (74, 3),
 (1, 2),
 (81, 2),
 (70, 2),
 (73, 1),
 (78, 1),
 (75, 1),
 (80, 1)]

In [ ]:
# Encode train label

from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit(train_labels)
encoded_train_labels = le.transform(train_labels)
encoded_valid_labels = le.transform(valid_labels)
encoded_test_labels = le.transform(test_labels)

In [ ]:
# Tokens IDs tensor

def encoder_generator(sentences,labels):

    sent_index = []
    input_ids = []
    attention_masks =[]

    for index,sent in enumerate(sentences):

        sent_index.append(index)

        encoded_dict = tokenizer.encode_plus(sent,
                                             add_special_tokens=True,
                                             max_length=50,
                                             padding='max_length',
                                             truncation = True,
                                             return_attention_mask=True,
                                             return_tensors='pt')
        input_ids.append(encoded_dict['input_ids'])

        attention_masks.append(encoded_dict['attention_mask'])

    input_ids = torch.cat(input_ids,dim=0).cuda()
    attention_masks = torch.cat(attention_masks,dim=0).cuda()
    labels = torch.tensor(labels).cuda()
    sent_index = torch.tensor(sent_index).cuda()

    return sent_index,input_ids,attention_masks,labels

train_sent_index,train_input_ids,train_attention_masks,train_encoded_label_tensors = encoder_generator(train_sentences,encoded_train_labels)
valid_sent_index,valid_input_ids,valid_attention_masks,valid_encoded_label_tensors = encoder_generator(valid_sentences,encoded_valid_labels)
print('Original: ', train_sentences[0])
print('Token IDs:', train_input_ids[0])

Original:  jesús ángel rojo “¿holocausto azteca mel gibson se quedó corto en los horrores de ‘apocalypto’
Token IDs: tensor([    4,  7261, 30934,  2165, 12780,  7816,     3,  1067, 22291, 28271,
         4030,  6841, 20560, 17610, 10045,  1022,  1062,  6534,  6726,  1036,
         1065, 15119,  1019,  1008,     3,  2059,  2726, 30950,  6443,     3,
            5,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1],
       device='cuda:0')


In [ ]:
# Connvert train, dev input by using TensorDataset

from torch.utils.data import TensorDataset,random_split

train_dataset = TensorDataset(train_input_ids,train_attention_masks,train_encoded_label_tensors)
valid_dataset = TensorDataset(valid_input_ids,valid_attention_masks,valid_encoded_label_tensors)

print('train data samples is {}'.format(len(train_dataset)))
print("valid data samples is {}".format(len(valid_dataset)))

train data samples is 11413
valid data samples is 2854


In [ ]:
# Set cuda by using device

from torch.utils.data import DataLoader,RandomSampler,SequentialSampler

bs=128

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_data_loader = DataLoader(train_dataset,
                              sampler=RandomSampler(train_dataset),
                              batch_size=bs)
valid_data_loader = DataLoader(valid_dataset,
                              sampler=RandomSampler(valid_dataset),
                              batch_size=bs)

Load model BETO

In [ ]:
from transformers import AutoModel

beto = AutoModel.from_pretrained('dccuchile/bert-base-spanish-wwm-cased')
beto = beto.to(device)

Some weights of BertModel were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Build CNN

In [ ]:
# Build class CNN

import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoModel

class CNN(nn.Module):
    def __init__(self, embedding_dim, n_filters, filter_sizes, output_dim,
                 dropout, pad_idx):

        super().__init__()

        self.fc_input = nn.Linear(embedding_dim,embedding_dim)

        self.conv_0 = nn.Conv1d(in_channels = embedding_dim,
                                out_channels = n_filters,
                                kernel_size = filter_sizes[0])

        self.conv_1 = nn.Conv1d(in_channels = embedding_dim,
                                out_channels = n_filters,
                                kernel_size = filter_sizes[1])

        self.conv_2 = nn.Conv1d(in_channels = embedding_dim,
                                out_channels = n_filters,
                                kernel_size = filter_sizes[2])

        self.conv_3 = nn.Conv1d(in_channels = embedding_dim,
                                out_channels = n_filters,
                                kernel_size = filter_sizes[3])

        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self, encoded):

        #embedded = [batch size, sent len, emb dim]
        embedded = self.fc_input(encoded)
        #print(embedded.shape)

        embedded = embedded.permute(0, 2, 1)
        #print(embedded.shape)

        #embedded = [batch size, emb dim, sent len]

        conved_0 = F.relu(self.conv_0(embedded))
        conved_1 = F.relu(self.conv_1(embedded))
        conved_2 = F.relu(self.conv_2(embedded))
        conved_3 = F.relu(self.conv_3(embedded))

        #conved_n = [batch size, n_filters, sent len - filter_sizes[n] + 1]

        pooled_0 = F.max_pool1d(conved_0, conved_0.shape[2]).squeeze(2)
        pooled_1 = F.max_pool1d(conved_1, conved_1.shape[2]).squeeze(2)
        pooled_2 = F.max_pool1d(conved_2, conved_2.shape[2]).squeeze(2)
        pooled_3 = F.max_pool1d(conved_3, conved_3.shape[2]).squeeze(2)

        #pooled_n = [batch size, n_fibatlters]

        cat = self.dropout(torch.cat((pooled_0, pooled_1, pooled_2, pooled_3), dim = 1).cuda())

        #cat = [batch size, n_filters * len(filter_sizes)]

        result =  self.fc(cat)

        #print(result.shape)

        return result

In [ ]:
# Hyperparameters

EMBEDDING_DIM = 768
N_FILTERS = 32
FILTER_SIZES = [1,2,3,5]
OUTPUT_DIM = len(le.classes_)
DROPOUT = 0.2
PAD_IDX = tokenizer.pad_token_id

cnn = CNN(EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)
cnn = cnn.to(device)

In [ ]:
cnn

CNN(
  (fc_input): Linear(in_features=768, out_features=768, bias=True)
  (conv_0): Conv1d(768, 32, kernel_size=(1,), stride=(1,))
  (conv_1): Conv1d(768, 32, kernel_size=(2,), stride=(1,))
  (conv_2): Conv1d(768, 32, kernel_size=(3,), stride=(1,))
  (conv_3): Conv1d(768, 32, kernel_size=(5,), stride=(1,))
  (fc): Linear(in_features=128, out_features=2, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)

In [ ]:
0.00002 == 2e-5

True

In [ ]:
0.00001 == 19e-5

False

In [ ]:
# Optimizer and criterion

import torch.optim as optim

model_parameters = list(beto.parameters())+list(cnn.parameters())

optimizer = optim.Adam(model_parameters,lr=1e-5,eps=1e-8)
# optimizer = optim.Adam(model_parameters,lr=1e-3,eps=1e-8)
criterion = nn.CrossEntropyLoss()
criterion = criterion.to(device)

In [ ]:
# Calculate accuracy per batch during train

def categorical_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    max_preds = preds.argmax(dim = 1, keepdim = True) # get the index of the max probability
    correct = max_preds.squeeze(1).eq(y)
    return correct.sum() / torch.FloatTensor([y.shape[0]]).cuda()

In [ ]:
# Def for training

from tqdm import tqdm

def train():

    epoch_loss = 0
    epoch_acc = 0

    beto.train()
    cnn.train()

    for batch in tqdm(train_data_loader):
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        optimizer.zero_grad()

        embedded = beto(b_input_ids,b_input_mask)[0]

        predictions = cnn(embedded)

        loss = criterion(predictions, b_labels)

        acc = categorical_accuracy(predictions, b_labels)

        loss.backward()

        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()

    return epoch_loss / len(train_data_loader), epoch_acc / len(train_data_loader)

In [ ]:
# Class for predict label

import numpy as np

def predictions_labels(preds,labels):
    pred = np.argmax(preds,axis=1).flatten()
    label = labels.flatten()
    return pred,label

In [ ]:
# Evaluate loss, acc  and f1-macro

from sklearn.metrics import classification_report,accuracy_score,f1_score
def eval():
    epoch_loss = 0

    total_predictions = []
    total_true = []

    all_true_labels = []
    all_pred_labels = []

    beto.eval()
    cnn.eval()

    with torch.no_grad():

        for batch in tqdm(valid_data_loader):
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)

            embedded = beto(b_input_ids,b_input_mask)[0]
            predictions = cnn(embedded)

            loss = criterion(predictions, b_labels)
            epoch_loss += loss.item()

            predictions = predictions.detach().cpu().numpy()

            label_ids = b_labels.to('cpu').numpy()

            pred,true = predictions_labels(predictions,label_ids)

            all_pred_labels.extend(pred)
            all_true_labels.extend(true)

    print(classification_report(all_pred_labels,all_true_labels))
    avg_val_accuracy = accuracy_score(all_pred_labels,all_true_labels)
    macro_f1_score = f1_score(all_pred_labels,all_true_labels,average='macro')

    avg_val_loss = epoch_loss/len(valid_data_loader)

    print("accuracy = {0:.2f}".format(avg_val_accuracy))

    return avg_val_loss,avg_val_accuracy,macro_f1_score

In [ ]:
# Time for training

import time
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
# # Set device and gpu

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# n_gpu = torch.cuda.device_count()
# torch.cuda.get_device_name(0)

# beto.cuda()

# Training

In [ ]:
tname = "/content/beto_cnn_model_home"

In [ ]:
# epochs = 20
# epochs = 10
epochs = 5

best_macro_f1 = float('0')

for epoch in range(epochs):

    start_time = time.time()
    train_loss,train_acc = train()
    valid_loss,valid_acc,macro_f1 = eval()
    end_time = time.time()


    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if macro_f1 > best_macro_f1:
        best_macro_f1 = macro_f1
        torch.save(beto,f'{tname}_part1_train.pt')
        torch.save(cnn,f'{tname}_part2_train.pt')
        print("MODEL SAVED")

    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. acc: {valid_acc*100:.2f}%')
    print(f'\t Val. F1: {macro_f1*100:.2f}%')
    print('=============Epoch Ended==============\n')

100%|██████████| 23/23 [00:07<00:00,  3.23it/s]


              precision    recall  f1-score   support

           0       0.93      0.87      0.90      2506
           1       0.38      0.55      0.45       348

    accuracy                           0.83      2854
   macro avg       0.65      0.71      0.67      2854
weighted avg       0.87      0.83      0.85      2854

accuracy = 0.83
MODEL SAVED
Epoch: 01 | Epoch Time: 1m 29s
	Train Loss: 0.458 | Train acc: 81.53%
	 Val. Loss: 0.377 |  Val. acc: 83.39%
	 Val. F1: 67.43%
=============Epoch Ended==============



100%|██████████| 23/23 [00:07<00:00,  3.13it/s]


              precision    recall  f1-score   support

           0       0.95      0.89      0.92      2492
           1       0.47      0.66      0.55       362

    accuracy                           0.86      2854
   macro avg       0.71      0.78      0.74      2854
weighted avg       0.89      0.86      0.87      2854

accuracy = 0.86
MODEL SAVED
Epoch: 02 | Epoch Time: 1m 32s
	Train Loss: 0.335 | Train acc: 86.00%
	 Val. Loss: 0.320 |  Val. acc: 86.33%
	 Val. F1: 73.56%
=============Epoch Ended==============



100%|██████████| 23/23 [00:07<00:00,  3.13it/s]


              precision    recall  f1-score   support

           0       0.95      0.90      0.92      2482
           1       0.49      0.67      0.57       372

    accuracy                           0.87      2854
   macro avg       0.72      0.78      0.74      2854
weighted avg       0.89      0.87      0.87      2854

accuracy = 0.87
MODEL SAVED
Epoch: 03 | Epoch Time: 1m 33s
	Train Loss: 0.275 | Train acc: 88.82%
	 Val. Loss: 0.307 |  Val. acc: 86.62%
	 Val. F1: 74.34%
=============Epoch Ended==============



100%|██████████| 23/23 [00:07<00:00,  3.09it/s]


              precision    recall  f1-score   support

           0       0.91      0.92      0.92      2319
           1       0.64      0.61      0.63       535

    accuracy                           0.86      2854
   macro avg       0.78      0.77      0.77      2854
weighted avg       0.86      0.86      0.86      2854

accuracy = 0.86
MODEL SAVED
Epoch: 04 | Epoch Time: 1m 34s
	Train Loss: 0.221 | Train acc: 91.25%
	 Val. Loss: 0.321 |  Val. acc: 86.30%
	 Val. F1: 77.07%
=============Epoch Ended==============



100%|██████████| 23/23 [00:07<00:00,  3.07it/s]

              precision    recall  f1-score   support

           0       0.96      0.89      0.92      2512
           1       0.47      0.70      0.57       342

    accuracy                           0.87      2854
   macro avg       0.72      0.80      0.75      2854
weighted avg       0.90      0.87      0.88      2854

accuracy = 0.87
Epoch: 05 | Epoch Time: 1m 34s
	Train Loss: 0.171 | Train acc: 93.52%
	 Val. Loss: 0.350 |  Val. acc: 87.11%
	 Val. F1: 74.57%
=============Epoch Ended==============



In [ ]:
# Save BETO and CNN

torch.save(beto, tname + '_module_part1.pt')
torch.save(cnn,  tname + '_module_part2.pt')

In [ ]:
# # Save BETO and CNN
# tname = "/content/drive/MyDrive/TP1-AntiDiscriminator/weights/beto_cnn"
# torch.save(beto, tname + '_part1_train.pt')
# torch.save(cnn,  tname + '_part2_train.pt')

## EVALUATING

In [ ]:
# Load phobert and cnn

import torch
beto = torch.load(f'{tname}_part1_train.pt')
cnn = torch.load(f'{tname}_part2_train.pt')
beto.eval()
cnn.eval()


CNN(
  (fc_input): Linear(in_features=768, out_features=768, bias=True)
  (conv_0): Conv1d(768, 32, kernel_size=(1,), stride=(1,))
  (conv_1): Conv1d(768, 32, kernel_size=(2,), stride=(1,))
  (conv_2): Conv1d(768, 32, kernel_size=(3,), stride=(1,))
  (conv_3): Conv1d(768, 32, kernel_size=(5,), stride=(1,))
  (fc): Linear(in_features=128, out_features=2, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)

Predict label from true label

In [ ]:
test_sent_index, test_input_ids, test_attention_masks, test_encoded_label_tensors = encoder_generator(test_sentences,encoded_test_labels)
test_dataset = TensorDataset(test_input_ids,test_attention_masks,test_encoded_label_tensors)

test_data_loader = DataLoader(test_dataset,
                              sampler=RandomSampler(test_dataset),
                              batch_size=bs)

all_pred_labels = []
all_true_labels = []

with torch.no_grad():
  for batch in tqdm(test_data_loader):
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device)

    embedded = beto(b_input_ids,b_input_mask)[0]
    predictions = cnn(embedded)


    predictions = predictions.detach().cpu().numpy()

    label_ids = b_labels.to('cpu').numpy()

    pred, true = predictions_labels(predictions, label_ids)

    all_pred_labels.extend(pred)
    all_true_labels.extend(true)

100%|██████████| 28/28 [00:09<00:00,  2.98it/s]


In [ ]:
# The final score in the test set (classification report)

#print(classification_report(all_pred_labels,all_true_labels, digits = 4))
print(classification_report(all_true_labels,all_pred_labels, digits = 4))

              precision    recall  f1-score   support

           0     0.9212    0.9205    0.9208      2932
           1     0.6342    0.6362    0.6352       635

    accuracy                         0.8699      3567
   macro avg     0.7777    0.7784    0.7780      3567
weighted avg     0.8701    0.8699    0.8700      3567



In [ ]:
# Confusion matrix in the test set

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(all_true_labels, all_pred_labels)
cm

array([[2699,  233],
       [ 231,  404]])